# SQL in Python

## Connecting to a DataBase
Using `sqlite3` to connect to an SQLite database in Python:

In [30]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
  con = None
  try:
    con = sqlite3.connect(path)
    print('Connection to SQLite DB successful.')
  except Error as e:
    print(f'The error \'{e}\' occurred.')
  
  # returns connection object
  # connection object used to define the cursor and subsequent query
  return con

In [31]:
# the database is created automatically if it doesn't already exist
# given that the folder path exists, otherwise it will throw an error
# NOTE: The path is relative 
con = create_connection('./_data/sm_app.sqlite')

Connection to SQLite DB successful.


## Creating Tables

To execute queries in SQLite, use `cursor.execute()`. In this section, a function called `execute_query()` will be defined to use this method. This function will accept the connection object created above as well as a query string, which will then be passed to `cursor.execute()`.

`execute()` can execute any query passed to it in the form of a string. This method will be used to create tables in this section. The same method can also be used to update and delete also. 

In [32]:
type(con)

sqlite3.Connection

In [33]:
def execute_query(connection, query):
  cursor = connection.cursor()
  try:
    cursor.execute(query)
    connection.commit()
    print('Query executed successfully.')
  except Error as e:
    print(f'The error \'{e}\' occurred.')

In [34]:
# the query to create the 'users' table
create_users_table = """ 
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  age INTEGER,
  gender TEXT,
  nationality TEXT
);
"""

In [35]:
execute_query(con, create_users_table)

Query executed successfully.


Next, a `posts` table will be created with a 1:many relationship with the `users` table.

To do this, the `user_id` defined in the `posts` table needs to be cross-referenced with the `id` values in the `users` table. This can be achieved by using the `FOREIGN KEY` keyword and defining the `REFERENCES` in the foreign table.

In [36]:
# the query to create the 'posts' table
create_posts_table = """ 
CREATE TABLE IF NOT EXISTS posts (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  title TEXT NOT NULL,
  description TEXT NOT NULL,
  user_id INTEGER NOT NULL,
  FOREIGN KEY (user_id) REFERENCES users (id)
)
"""

In [37]:
execute_query(con, create_posts_table)

Query executed successfully.


In [41]:
# the query to create the 'comments' table
create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

In [42]:
execute_query(con, create_comments_table)

Query executed successfully.


In [43]:
# the query to
create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

In [44]:
execute_query(con, create_likes_table)

Query executed successfully.


## Inserting Records

When creating `users` in the `users` table, the user `id` column is autogenerated and autoincremented.

In [45]:
create_users = """ 
INSERT INTO
  users (name, age, gender, nationality)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

In [46]:
# execute_query(con, create_users)

Query executed successfully.


It's important to mention that the `user_id` column of the `posts` table is a foreign key that references the `id` column of the `users` table. As such, the `user_id` column must contain a value that already exists in the `id` column of the `users` table. If it doesn't exist, the query will throw an error.

In [47]:
create_posts = """
INSERT INTO
  posts (title, description, user_id)
VALUES
  ("Happy", "I am feeling very happy today", 1),
  ("Hot Weather", "The weather is very hot today", 2),
  ("Help", "I need some help with my work", 2),
  ("Great News", "I am getting married", 1),
  ("Interesting Game", "It was a fantastic game of tennis", 5),
  ("Party", "Anyone up for a late-night party today?", 3);
"""


In [48]:
# execute_query(con, create_posts)

Query executed successfully.


In [49]:
create_comments = """
INSERT INTO
  comments (text, user_id, post_id)
VALUES
  ('Count me in', 1, 6),
  ('What sort of help?', 5, 3),
  ('Congrats buddy', 2, 4),
  ('I was rooting for Nadal though', 4, 5),
  ('Help with your thesis?', 2, 3),
  ('Many congratulations', 5, 4);
"""

In [50]:
# execute_query(con, create_comments)

Query executed successfully.


In [51]:
create_likes = """
INSERT INTO
  likes (user_id, post_id)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

In [52]:
# execute_query(con, create_likes)

Query executed successfully.


## Selecting Records

To select records using SQLite, the `cussor.execute()` can be used. However, `.fetchall()` will also need to called. This method will return a list of tuples representing the values of each row in the table.

In [53]:
def execute_read_query(connection, query):
  cursor = connection.cursor()
  result = None
  try:
    cursor.execute(query)
    result = cursor.fetchall()
    return result
  except Error as e:
    print(f'The error \'{e}\' occurred.')

In [54]:
select_users = "SELECT * FROM users"
users = execute_read_query(con, select_users)

In [55]:
for user in users:
  print(user)

(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')


## Updating Table Records

In [56]:
update_post_description = """
UPDATE
  posts
SET
  description = 'The weather has become pleasant now.'
WHERE
  id = 2
"""

In [57]:
# execute_query(con, update_post_description)

Query executed successfully.
